In [49]:
import Modules.SQcircuit_extensions as sq_ext
import Modules.figures as figs
import SQcircuit as sq
import numpy as np
import matplotlib.pyplot as plt
import importlib
from scipy.linalg import eigh
import qutip as qt
import sympy as sym
from joblib import Parallel, delayed, cpu_count

%matplotlib ipympl
plt.rcParams['text.usetex'] = False
importlib.reload(sq_ext)
importlib.reload(sq)
importlib.reload(figs)
np.set_printoptions(linewidth=300, formatter={'float': '{:.12f}'.format})

fF  = 1e-15
nH  = 1e-9
GHz = 1e9

In [50]:
Δ = 0.1
LF = 26.5
CF = 26.9
EJ = 5.4
ω_r = 6.46
LR = 140
CR = 1 / (LR * nH) / (ω_r * 2 * np.pi * GHz) ** 2 / fF
Lq, Lr = sq_ext.LF_LR_eff_to_Lq_Lr(LF=LF, LR=LR, Δ=Δ)
C, Csh, CJ = sq_ext.CF_CR_eff_to_C_CJ_Csh(CF=CF, CR=CR)
C, Csh, CJ 

(8.671159748277237, 11.282210062930691, 11.282210062930691)

In [51]:

Δ_prime = 0.1
LF_prime = 20.4
CF_prime = 22.4
EJ_prime = 9.5
ω_r_prime = 6.274
LR_prime = 100
# LR_prime = 190
CR_prime = 1 / (LR_prime * nH) / (ω_r_prime * 2 * np.pi * GHz) ** 2 / fF
Lq_prime, Lr_prime = sq_ext.LF_LR_eff_to_Lq_Lr(LF=LF_prime, LR=LR_prime, Δ=Δ_prime)
C_prime, Csh_prime, CJ_prime = sq_ext.CF_CR_eff_to_C_CJ_Csh(CF=CF_prime, CR=CR_prime)
C_prime, Csh_prime, CJ_prime 

(12.870079518472817, 7.982480120381795, 7.982480120381795)

In [53]:
nmax_r = 10
nmax_f = 10
n_eig = 5
φ_ext = 0.5

In [54]:
fluxonium_0       = sq_ext.sq_fluxonium(C_F_eff=CF,       L_F_eff=LF,       Δ=Δ,       EJ=EJ,       nmax_f=nmax_f, φ_ext=φ_ext)
resonator_0       = sq_ext.sq_resonator(C_R_eff=CR,       L_R_eff=LR,       Δ=Δ,       EJ=EJ,       nmax_r=nmax_r)
fluxonium_prime_0 = sq_ext.sq_fluxonium(C_F_eff=CF_prime, L_F_eff=LF_prime, Δ=Δ_prime, EJ=EJ_prime, nmax_f=nmax_f, φ_ext=φ_ext)
resonator_prime_0 = sq_ext.sq_resonator(C_R_eff=CR_prime, L_R_eff=LR_prime, Δ=Δ_prime, EJ=EJ_prime, nmax_r=nmax_r)
fluxonium_0      .diag(2); 
resonator_0      .diag(2);
fluxonium_prime_0.diag(2);
resonator_prime_0.diag(2);

In [55]:
E0_F  = fluxonium_0.efreqs[1]-fluxonium_0.efreqs[0] 
E0_R  = resonator_0.efreqs[1]-resonator_0.efreqs[0]
E0_Fp = fluxonium_prime_0.efreqs[1]-fluxonium_prime_0.efreqs[0]
E0_Rp = resonator_prime_0.efreqs[1]-resonator_prime_0.efreqs[0]
E0_F  , E0_R  , E0_Fp , E0_Rp 

(3.7002016310824453, 6.459999999999947, 3.450503873328403, 6.273999999999957)

In [56]:
ω0_FR   = np.abs( E0_F-E0_R )
ω0_FFp  = np.abs( E0_F-E0_Fp )
ω0_FRp  = np.abs( E0_F-E0_Rp )
ω0_RRp  = np.abs( E0_R-E0_Rp )
ω0_FpRp = np.abs( E0_Fp-E0_Rp )
ω0_FR , ω0_FFp, ω0_FRp, ω0_RRp, ω0_FpRp 

(2.7597983689175014,
 0.24969775775404246,
 2.573798368917512,
 0.18599999999998929,
 2.8234961266715546)

In [57]:
CC = 10
only_inner = True
compensate_extra_cap = False
only_renormalization = False
C_mat = sq_ext.C_mat_qubit_C_qubit(CC, CR, CF, CR_prime, CF_prime, only_inner, compensate_extra_cap, only_renormalization)
C_mat

array([[29.400000000000, -2.500000000000, 2.500000000000, 2.500000000000],
       [-2.500000000000, 6.835579874139, -2.500000000000, -2.500000000000],
       [2.500000000000, -2.500000000000, 24.900000000000, 2.500000000000],
       [2.500000000000, -2.500000000000, 2.500000000000, 8.935039759236]])

In [58]:
C_inv = np.linalg.inv(C_mat)
C_inv

array([[0.035582356454, 0.009879788319, -0.001912259437, -0.006656464140],
       [0.009879788319, 0.169350747891, 0.011864567222, 0.041299870042],
       [-0.001912259437, 0.011864567222, 0.042346438444, -0.007993700239],
       [-0.006656464140, 0.041299870042, -0.007993700239, 0.127573588565]])

In [68]:
CF_prime_tilde, CF_prime

(23.614736841006426, 22.4)

In [59]:
CF_tilde = C_inv[0, 0] ** -1
CR_tilde = C_inv[1, 1] ** -1
CF_prime_tilde = C_inv[2, 2] ** -1
CR_prime_tilde = C_inv[3, 3] ** -1
fluxonium       = sq_ext.sq_fluxonium(C_F_eff=CF_tilde,       L_F_eff=LF,       Δ=Δ,       EJ=EJ,       nmax_f=nmax_f, φ_ext=φ_ext)
resonator       = sq_ext.sq_resonator(C_R_eff=CR_tilde,       L_R_eff=LR,       Δ=Δ,       EJ=EJ,       nmax_r=nmax_r)
fluxonium_prime = sq_ext.sq_fluxonium(C_F_eff=CF_prime_tilde, L_F_eff=LF_prime, Δ=Δ_prime, EJ=EJ_prime, nmax_f=nmax_f, φ_ext=φ_ext)
resonator_prime = sq_ext.sq_resonator(C_R_eff=CR_prime_tilde, L_R_eff=LR_prime, Δ=Δ_prime, EJ=EJ_prime, nmax_r=nmax_r)
fluxonium      .diag(2); 
resonator      .diag(2);
fluxonium_prime.diag(2);
resonator_prime.diag(2);

In [60]:
# 0=F    1=R    2=Fp    3=Rp
C_FR_inv    = C_inv[0,1] * fF**-1
C_FFp_inv   = C_inv[0,2] * fF**-1
C_FRp_inv   = C_inv[0,3] * fF**-1
C_RRp_inv   = C_inv[1,3] * fF**-1
C_FpRp_inv  = C_inv[2,3] * fF**-1

In [61]:
qF  = np.abs( fluxonium       . charge_op(0,basis='eig')[0,1] )
qR  = np.abs( resonator       . charge_op(0,basis='eig')[0,1] )
qFp = np.abs( fluxonium_prime . charge_op(0,basis='eig')[0,1] )
qRp = np.abs( resonator_prime . charge_op(0,basis='eig')[0,1] )

In [62]:
E_Q_FR   = 0.5 * C_FR_inv   * qF  * qR  / 2 / np.pi / GHz
E_Q_FFp  = 0.5 * C_FFp_inv  * qF  * qFp / 2 / np.pi / GHz
E_Q_FRp  = 0.5 * C_FRp_inv  * qF  * qRp / 2 / np.pi / GHz
E_Q_RRp  = 0.5 * C_RRp_inv  * qR  * qRp / 2 / np.pi / GHz
E_Q_FpRp = 0.5 * C_FpRp_inv * qFp * qFp / 2 / np.pi / GHz

In [63]:
E_Q_FR  , E_Q_FFp , E_Q_FRp , E_Q_RRp , E_Q_FpRp

(0.14171599069701885,
 -0.04201840693620296,
 -0.1114816556240867,
 0.39403957655833266,
 -0.15328283564673806)

In [64]:
E_F = fluxonium.efreqs[1] - fluxonium.efreqs[0] 
E_R = resonator.efreqs[1] - resonator.efreqs[0]
E_Fp = fluxonium_prime.efreqs[1] - fluxonium_prime.efreqs[0]
E_Rp = resonator_prime.efreqs[1] - resonator_prime.efreqs[0]
E_F, E0_R, E0_Fp, E0_Rp

(3.6027182180042123, 6.459999999999947, 3.450503873328403, 6.273999999999957)

In [65]:
ω_FR = np.abs(E0_F - E0_R)
ω_FFp = np.abs(E0_F - E0_Fp)
ω_FRp = np.abs(E0_F - E0_Rp)
ω_RRp = np.abs(E0_R - E0_Rp)
ω_FpRp = np.abs(E0_Fp - E0_Rp)
ω_FR, ω0_FFp, ω0_FRp, ω0_RRp, ω0_FpRp

(2.7597983689175014,
 0.24969775775404246,
 2.573798368917512,
 0.18599999999998929,
 2.8234961266715546)